建物データと浸水面標高ラスターデータから建物の被災データを生成。（必要手続き：Driveの接続）

integratinonKeyとprojectIDが共に設定されている場合、Re:EarthCMSへのアップロードを行います。最後に出力される各種URLを、Re:Earthの編集画面に入力してください。

In [ ]:
# @title Googleドライブの準備
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
# @title 設定するパラメータ

# ケース名：解析を通して使います。
casename = "omuta-r2-giaj" #"omuta-r2-giaj" #-giaj" #"kurume-r2" #"kurume-r2-GIAJ"
file_uploading = "heatmap_all_mokuzou.zip"

obsdate = "2020-07-07" #"2020-07-09" #"2020-07-10"
satellite = "国土地理院" #"ASNARO-2" #"国土地理院" #"Sentinel-1"

uploadlist = [
      #"浸水域・浸水深（3Dタイル）",
      #"浸水域・浸水深（画像タイル）",
      "建物被災状況（CSVファイル）",
      #"被災建物・ヒートマップ画像タイル：床上（木造）",
      #"被災建物・ヒートマップ画像タイル：床下（木造）",
      "被災建物・ヒートマップ画像タイル:木造家屋総数"
]

# Re:Earthへのアップロード用
# 片方でも None の場合はアップロードされません。
integrationKey = None # インテグレーションキー（文字列）
projectId = None # プロジェクトページのURLより（文字列）


In [ ]:
# @title 詳細設定用パラメータ（基本的には操作不要）

enable_savefig = False # 図をGoogleドライブに保存する？

zoomlevel = 15 # DEM zoom level to fetch

# デバッグ用出力
global globalflag_debug

# 保存先
path_home = "/content/drive/MyDrive/plateau-2023-uc01-satellite-analytics/PLATEAU-FloodSAR/"
#path_cgml = path_home + "/CityGML/"
path_dem = path_home + "/DEM/"
path_case = path_home + casename + "/"
path_upload = path_case + "ForUpload/"
file_fldbbox = path_case + "floodprb_bbox.npy"
json_boundary = path_case +"boundary.json"
file_bbox = path_case + "boundbox.npy"

file_flooddem = path_case + "flood_dem_{:04d}.npz"
file_flooddepth = path_case + "flood_depth_{:04d}.npz"
file_building_csv = path_upload + "buildings.csv"

path_flood3Dtile_tmp =  "flood3dtile_depth/"
path_deltphtile_tmp = "ellipsoid_xyztiles/"
file_flood3Dtile_zip = path_upload+"3dtiles.zip"
file_floodXYZtile_zip = path_upload+"xyztile.zip"
file_floodHeattile_yukauemokuzou_zip = path_upload+"heatmap_yukaue_mokuzou.zip"
file_floodHeattile_yukashitamokuzou_zip = path_upload+"heatmap_yukashita_mokuzou.zip"
file_floodHeattile_allmokuzou_zip = path_upload+"heatmap_all_mokuzou.zip"

In [ ]:
#!pip list

In [ ]:
#@title ライブラリインポート
!pip install plateauutils
!pip install reearthcmsapi
!pip install --upgrade numpy
!pip install --upgrade scipy
!pip install --upgrade scikit-image

import matplotlib.pyplot as plt
import numpy as np
from shapely import box
#from scipy import interpolate
from progressbar import progressbar
import zipfile
import os
import sys
import shutil
import plateauutils
import reearthcmsapi
from plateauutils.flood_converter.flood_to_3dtiles import FloodTo3dtiles
from plateauutils.flood_converter.flood_to_png import FloodToPng
from reearthcmsapi.apis.tags import items_api
from reearthcmsapi.model.versioned_item import VersionedItem
from reearthcmsapi.model.asset_embedding import AssetEmbedding
from reearthcmsapi.apis.tags import assets_project_api
from reearthcmsapi.model.asset import Asset
from pprint import pprint
sys.path.append(path_home)
import plateau_floodsar_lib as pfsl

## Re:EarthCMSへのアップロード

In [ ]:
#@title Re:Earth CMSへのアップロード
if integrationKey is not None and projectId is not None:
  configuration = reearthcmsapi.Configuration(
      host = "https://api.cms.test.reearth.dev/api",
      access_token = integrationKey
  )

  client = reearthcmsapi.ApiClient(configuration)

  api_client = reearthcmsapi.ApiClient(configuration)
  api_instance = assets_project_api.AssetsProjectApi(api_client)

  # Create an instance of the API class
  api_instance = assets_project_api.AssetsProjectApi(api_client)
  path_params = {
      'projectId': projectId,
  }

  detaillist = {
      "浸水域・浸水深（3Dタイル）": [file_floodXYZtile_zip, "浸水域・浸水深（3Dタイル）\n{ff}/tileset.json"],
      "浸水域・浸水深（画像タイル）":[file_floodXYZtile_zip, "浸水域・浸水深（画像タイル）\n{ff}/{z}/{x}/{y}.png"],
      "建物被災状況（CSVファイル）":[file_building_csv, "建物被災状況（CSVファイル）\n{ff}.csv"],
      "被災建物・ヒートマップ画像タイル：床上（木造）":[file_floodHeattile_yukauemokuzou_zip,"被災建物・ヒートマップ画像タイル：床上（木造）\n{ff}/{z}/{x}/{y}.png"],
      "被災建物・ヒートマップ画像タイル：床下（木造）":[file_floodHeattile_yukashitamokuzou_zip,"被災建物・ヒートマップ画像タイル：床下（木造）\n{ff}/{z}/{x}/{y}.png"],
      "被災建物・ヒートマップ画像タイル:木造家屋総数":[file_floodHeattile_allmokuzou_zip,"被災建物・ヒートマップ画像タイル:木造家屋総数\n{ff}/{z}/{x}/{y}.png"]
  }

  outtxt = ""
  for key in uploadlist:
    if key not in detaillist:
      continue
    ff, txt = detaillist[key]
    if not os.path.exists(ff):
      continue
    body = dict(
        file=open(ff, 'rb'),
        skip_decompression=False,
    )
    try:
        # Create an new asset.
        api_response = api_instance.asset_create(
            path_params=path_params,
            body=body,
        )
        #pprint(api_response)
        outtxt += txt.format(ff=api_response.body["url"][:-4],x="{x}",y="{y}",z="{z}")+"\n\n"
    except reearthcmsapi.ApiException as e:
        print("Exception when calling AssetsProjectApi->asset_create: %s\n" % e)

  print("以下の情報をRe:Earth編集画面に入力してください。")
  print("------------------------------------------------")
  print(f"観測日: {obsdate}")
  print(f"観測衛星: {satellite}")
  print()
  print(outtxt)
else:
  print("Re:Earth CMSへのアップロードは行われませんでした。")

以下の情報をRe:Earth編集画面に入力してください。
------------------------------------------------


